<a href="https://colab.research.google.com/github/anko191/Python_Kaggle/blob/master/Tensorflow/For_gaining_scores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 予測値をp_min ~ p_max の間に落とし込む

In [ ]:
p_min = 0.001
p_max = 0.999

In [ ]:
def metric(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred, p_min, p_max)
    return -K.mean(y_true * K.log(y_pred) + (1 - y_true) * K.log(1 - y_pred))

## BinaryCrossentropyのlabel_smoothingを使う
* import tensorflow.keras.losses as LOSS

In [ ]:
model.compile(optimizer = optimizer,
                loss = LOSS.BinaryCrossentropy(label_smoothing = 0.001),
                metrics = metric,
                )